## Visualization of Robot missions

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import geopandas
import contextily as cx
import branca
import folium
import branca.colormap as cm
import matplotlib.pyplot as plt
from selenium import webdriver
import time 
import geopandas
import haversine as hs
from haversine import Unit
import pickle

module_path = str(Path.cwd().parents[0] / "src")
#module_path = str(Path.cwd(__file__).parents[0] / "py")

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
input_data_folder_base = Path("../results/")
input_data_folder = Path.cwd() / input_data_folder_base
overall_collection_file_name = "overall_collection_2023.p"
key_parameter_file_name = "key_parameter_2023.p"
cluster_positions_file_name = "cluster_positions_2023.p"
print(f"Reading raw pandas files from \n    {input_data_folder}")

output_folder_base = Path("../html/")
output_image_folder = output_folder_base / Path("./images/")
print(f"Writing website information to \n    {output_folder_base}")

In [ ]:
df_results = pd.read_pickle(input_data_folder / overall_collection_file_name)
df_results.head(3)


In [ ]:
cluster_position_file = input_data_folder / Path(cluster_positions_file_name)

with open(cluster_position_file, 'rb') as fp:
    print(fp)
    centroids = pickle.load(fp)

centroids

In [ ]:
for location_name in df_results.experiment_location.unique():

    print(f"{location_name} ...")
    df_sep_pos = df_results[(df_results.experiment_location == location_name)]

    print(f"    ... providing key data for tables")
    df_key_param = df_sep_pos[df_sep_pos.meas_running].reset_index()\
            .groupby('corresponding_meas_file')\
                .agg(start=('datetime_AMT', 'min'),
                     end=('datetime_AMT', 'max'),
                     experiment_location = ('experiment_location', 'first'),
                     position = ('position', 'first'),
                     lat_est_mean=('lat_est', 'mean'),
                     lon_est_mean=('lon_est', 'mean'),
                     CO2_min=('CO2(ppm)', 'min'),
                     CO2_max=('CO2(ppm)', 'max'),
                     humidity_in_mean=('humidity in(rH)', 'mean'),
                     humidity_out_mean=('humidity out(rH)', 'mean'),
                     airtemp_in_mean=('airtemp in(degreeC)', 'mean'),
                     PAR_in_mean=('PAR(umol m-2s-1)', 'mean'),
                     airtemp_out_mean=('airtemp out(degreeC)', 'mean'))\
            .sort_values(by=['position', 'start'])
    
    print(f"    ... generating map background")
    map = folium.Map(location = [df_sep_pos.lat_est.median(),  
                                 df_sep_pos.lon_est.median()], 
                                 tiles='Stamen Terrain', zoom_start = 16, max_zoom=23)

    basemaps = {
        'Google Maps': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Maps',
            overlay = True,
            control = True
        ),
        'Google Satellite': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Satellite',
            overlay = True,
            control = True
        ),
        'Google Terrain': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Terrain',
            overlay = True,
            control = True
        ),
        'Google Satellite Hybrid': folium.TileLayer(
            tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
            attr = 'Google',
            name = 'Google Satellite',
            overlay = True,
            control = True
        ),
        'Esri Satellite': folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri',
            name = 'Esri Satellite',
            overlay = True,
            control = True
        )
    }

    # Add custom basemaps
    #basemaps['Google Maps'].add_to(map)
    basemaps['Google Satellite Hybrid'].add_to(map)

    for index, row in df_sep_pos.iterrows():
        map.add_child(folium.CircleMarker(location = [row['lat_est'],row['lon_est']],
                                        color = 'gray',
                                        radius = 1, fill=True))
        
    for index, row in df_sep_pos[df_sep_pos.meas_running].iterrows():
        map.add_child(folium.CircleMarker(location = [row['lat_est'],row['lon_est']],
                                        color = 'red',
                                        radius = 1, fill=True))

    table_params = ['start', 'CO2_min', 'CO2_max', 'airtemp_in_mean', 'PAR_in_mean', 'position']

    print(f"    ... providing interactive elements")
    for index, location_cords in enumerate(centroids[location_name]):

        html_data = df_key_param[df_key_param.position== index][table_params]\
                        .reset_index()\
                        .sort_values(by=['position', 'start'])\
                        .to_html(classes='table table-stripped', 
                                index=False, float_format="%.2f")
        html = folium.Html(
            f"""
            <!DOCTYPE html>
            <html>
            <h5 align="center" style="color:black"><strong><u>Position {index} {location_name}</u><strong>
            </h5> 
            {html_data}
            <img src="./images/Balbina_Positions_{index}.png" alt="HTML5 Icon" style="width:500px;height:500px;"> 
            </html>
            """, 
            script=True)

        popup  = folium.Popup(html, max_width=1000, max_height=1000)
        folium.vector_layers.Marker(location=[location_cords[0], location_cords[1]], 
                                    tooltip=f'Position {index}', popup = popup).add_to(map)

    map

    filename = f"{location_name}.html"
    html_folder = Path("../html/")
    html_file = Path.cwd() / html_folder / Path(filename)  
    print(f"    ... saving to \n        {html_file}")
        
    map.save(html_file)
